In [2]:
import numpy 
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import remove_self_loops, add_self_loops, add_remaining_self_loops, degree

In [3]:
%run graph.ipynb

Data(x=[734160, 106], edge_index=[2, 5873260], y=[734160], norm=[6607420])
tensor(indices=tensor([[721050, 721050, 721050,  ..., 734157, 734158, 734159],
                       [721051, 721052, 721053,  ..., 734157, 734158, 734159]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(734160, 734160), nnz=6607420, layout=torch.sparse_coo)


In [5]:
print(data)

Data(x=[5139120, 107], edge_index=[2, 41112940], norm=[46252060])


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch_geometric.data import Data

# Graph Convolution Layer
class GraphConvolution(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GraphConvolution, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.weight = nn.Parameter(torch.Tensor(input_dim, output_dim))
        self.bias = nn.Parameter(torch.Tensor(output_dim))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, x, adj):
        support = torch.matmul(x, self.weight)
        output = torch.sparse.mm(adj, support)
        output = output + self.bias
        return F.relu(output)

# GCN Model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(input_dim, hidden_dim)
        self.gc2 = GraphConvolution(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, adj):
        x = self.gc1(x, adj)
        x = self.dropout(x)
        x = self.gc2(x, adj)
        # print(x.shape)
        return x


In [8]:
def train_gcn_model(model, data, adj_matrix, epochs=20, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.MSELoss()

    for epoch in range(epochs):
        optimizer.zero_grad()
        output = model(data.x, adj_matrix)
        print(output)
        print("-------")
        print(data.y)
        loss = criterion(output.view(-1), data.y)  # Flatten output for regression
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch + 1}, Loss: {loss.item()}')


gcn_model = GCN(input_dim=data.x.size(1), hidden_dim=16, output_dim=1)

# Train GCN model for regression
train_gcn_model(gcn_model, data, adj_matrix, epochs=10)

tensor([[0.0000e+00],
        [1.9706e+16],
        [1.6043e+17],
        ...,
        [       nan],
        [       nan],
        [       nan]], grad_fn=<ReluBackward0>)
-------
tensor([0., 0., 0.,  ..., 0., 0., 0.])
Epoch 1, Loss: nan
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<ReluBackward0>)
-------
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<ReluBackward0>)
-------
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<ReluBackward0>)
-------
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<ReluBackward0>)
-------
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([[nan],
        [nan],
        [nan],
        ...,
 